In [76]:
import numpy as np
import tensorflow as tf
import pandas as pd

# define paramter 
num_vectors = 1000
num_clusters = 4
num_steps = 100

# inital data strcture
x_values = []
y_values = []
vector_values = []

# create train data
for i in range(num_vectors):
    if np.random.random() > 0.5:
        x_values.append(np.random.normal(0.4, 0.7))
        y_values.append(np.random.normal(0.2, 0.8))
    else:
        x_values.append(np.random.normal(0.6, 0.4))
        y_values.append(np.random.normal(0.8, 0.5))

# combine x_valus and y_values to be a tuple        
vector_values = zip(x_values, y_values)
vectors = tf.constant(list(vector_values))

# initial centroid
n_sample = tf.shape(vectors)[0]
random_indices = tf.random_shuffle(tf.range(0, n_sample))
begin = [0,]
size = [num_clusters, ]
centroid_indices = tf.slice(random_indices, begin, size)
centroids = tf.Variable(tf.gather(vectors, centroid_indices))

# cost function, tf.expand_dims : 延伸向量的維度, 0加在前面, 1加在後面
expanded_vectors = tf.expand_dims(vectors, 0)
expanded_centroids = tf.expand_dims(centroids, 1)
vectors_subtration = tf.sub(expanded_vectors, expanded_centroids)
eucliden_distances = tf.reduce_sum(tf.square(vectors_subtration), 2)
assignments = tf.to_int32(tf.argmin(eucliden_distances, 0))

# tf.dynamic_partition : 將索引（assignments）的結果分類到各個群（num_clusters）之中
partitions = tf.dynamic_partition(vectors, assignments, num_clusters)

# update centroid, tf.concat : 將指定維度的向量合併在一起
update_centroids = tf.concat(0, [tf.expand_dims(tf.reduce_mean(partition, 0), 0) for partition in partitions])

In [78]:
# display result 
import matplotlib.pyplot as plt
def display_partition(x_values, y_values, assignments_values):
    labels = []
    colors = ["red", "blue", "green", "yellow"]
    for i in range(len(assignments_values)):
        labels.append(colors[assignments_values[i]])
    color = labels
    df = pd.DataFrame(dict(x = x_values, y = y_values, color = labels))
    fig, ax = plt.subplots()
    ax.scatter(df['x'], df['y'], c=df['color'])
    plt.show()

In [82]:
# run
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
for step in range(num_steps):
    _, centroid_values, assignments_values = sess.run([update_centroids, centroids, assignments])
plt.plot(x_values, y_values, 'o', label = 'Input Data')
plt.legend()
plt.show()
display_partition(x_values, y_values, assignments_values)